In [ ]:
from pyomo.environ import *
import pandas as pd

# Read data from CSV files ADJUSTED THE DATA LOADS TO WORK
fixed_cost_production = pd.read_csv("/content/drive/MyDrive/Thesis/ProblemData/MIP/MIP2/fixed_cost_production.csv")
fixed_cost_production.index += 1
fixed_cost_production = fixed_cost_production.drop("Unnamed: 0", axis = 1)
fixed_cost_production.columns = fixed_cost_production.columns.astype(int)

variable_cost_production = pd.read_csv("/content/drive/MyDrive/Thesis/ProblemData/MIP/MIP2/variable_cost_production.csv")
variable_cost_production.index += 1
variable_cost_production = variable_cost_production.drop("Unnamed: 0", axis = 1)
variable_cost_production.columns = variable_cost_production.columns.astype(int)

variable_cost_storage = pd.read_csv("/content/drive/MyDrive/Thesis/ProblemData/MIP/MIP2/variable_cost_storage.csv")
variable_cost_storage.index += 1
variable_cost_storage = variable_cost_storage.drop("Unnamed: 0", axis = 1)
variable_cost_storage.columns = variable_cost_storage.columns.astype(int)

demand = pd.read_csv("/content/drive/MyDrive/Thesis/ProblemData/MIP/MIP2/demand.csv")
demand.index += 1
demand = demand.drop("Unnamed: 0", axis = 1)
demand.columns = demand.columns.astype(int)

revenue = pd.read_csv("/content/drive/MyDrive/Thesis/ProblemData/MIP/MIP2/revenue.csv")
revenue.index += 1
revenue = revenue.drop("Unnamed: 0", axis = 1)
revenue.columns = revenue.columns.astype(int)

# Define the model
model = ConcreteModel()

# Define the sets
model.T = RangeSet(1, 12)  # Time periods
model.P = Set(initialize=[1, 2, 3, 4, 5])  # Products

# Define parameters
model.fixed_cost = Param(model.P, model.T, initialize=fixed_cost_production.stack().to_dict())
model.variable_cost_production = Param(model.P, model.T, initialize=variable_cost_production.stack().to_dict())
model.variable_cost_storage = Param(model.P, model.T, initialize=variable_cost_storage.stack().to_dict())
model.demand = Param(model.P, model.T, initialize=demand.stack().to_dict())
model.revenue = Param(model.P, model.T, initialize=revenue.stack().to_dict())
model.storage_capacity = Param(model.P, initialize={1: 580,
    2: 687,
    3: 599,
    4: 788,
    5: 294
})
model.production_capacity = Param(model.P, initialize={1: 1080,
    2: 908,
    3: 408,
    4: 1000,
    5: 403
})

# Define the decision variables
model.production = Var(model.P, model.T, domain=NonNegativeReals)
model.storage = Var(model.P, model.T, domain=NonNegativeReals)
model.sales = Var(model.P, model.T, domain=NonNegativeReals)

# Define binary variables to indicate production activity MODIFIED ADDED BINARY VARIABLE FOR LOGIC IN OBJECTIVE FUNCTION
model.Y = Var(model.P, model.T, domain=Binary)

#set domain to integer for food MODIFIED ADDED INTEGRALITY HERE INSTEAD OF CONSTRAINT
for t in model.T:
    model.production[2, t].domain = NonNegativeIntegers
    model.production[1, t].domain = NonNegativeIntegers
    model.storage[2, t].domain = NonNegativeIntegers
    model.storage[1, t].domain = NonNegativeIntegers
    model.sales[2, t].domain = NonNegativeIntegers
    model.sales[1, t].domain = NonNegativeIntegers

# Define the objective function
def objective_rule(model):
    return sum(model.revenue[p, t] * model.sales[p, t] -
               model.fixed_cost[p, t] * model.Y[p, t] - # MODIFIED: REMOVE INEQUALITY
               model.variable_cost_production[p, t] * model.production[p, t] -
               model.variable_cost_storage[p, t] * model.storage[p, t]
               for p in model.P for t in model.T)
model.objective = Objective(rule=objective_rule, sense=maximize)

# Define the constraints
def inventory_balance_rule(model, p, t):
    if t == 1:
        return model.storage[p, t] == model.production[p, t] - model.sales[p, t]
    else:
        return model.storage[p, t] == model.storage[p, t-1] + model.production[p, t] - model.sales[p, t]
model.inventory_balance = Constraint(model.P, model.T, rule=inventory_balance_rule)

def demand_rule(model, p, t):
    return model.sales[p, t] <= model.demand[p, t]
model.demand_constraint = Constraint(model.P, model.T, rule=demand_rule)

def storage_capacity_rule(model, p, t):
    return model.storage[p, t] <= model.storage_capacity[p]
model.storage_capacity_constraint = Constraint(model.P, model.T, rule=storage_capacity_rule)

# Ensure Y is 1 if P is positive MODIFIED: ADDED BINARY CONSTRAINT
def ensure_binary_rule(model, p, t):
    return model.production[p, t] <= model.Y[p, t] * model.production_capacity[p]

model.EnsureBinary = Constraint(model.P, model.T, rule=ensure_binary_rule)

# Solve the model
solver = SolverFactory('gurobi')
results = solver.solve(model)

# Print results
print("Objective function value:", model.objective())
print()
for t in model.T:
    for i in model.P:
        print(f"X[{i}, {t}]: {model.production[i, t].value}, Y[{i}, {t}]: {model.Y[i, t].value}, S[{i}, {t}]: {model.storage[i, t].value}")